In [ ]:
!nvidia-smi

Wed Jan  5 16:06:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/FineTuningSentenceBert

/content/drive/MyDrive/FineTuningSentenceBert


In [ ]:
pip install sentence_transformers


     |████████████████████████████████| 78 kB 5.2 MB/s 
     |████████████████████████████████| 3.4 MB 17.8 MB/s 
     |████████████████████████████████| 6.8 MB 43.6 MB/s 
     |████████████████████████████████| 1.2 MB 58.5 MB/s 
     |████████████████████████████████| 61 kB 696 kB/s 
     |████████████████████████████████| 596 kB 67.0 MB/s 
     |████████████████████████████████| 3.3 MB 59.5 MB/s 
     |████████████████████████████████| 895 kB 60.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=c15cfc35f4e1ccc45e33cd4b4163ce9511d2b69d0e6b1b1cb9a9349bbe731078
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Define a list with sentences (1k - 100k sentences)
import re
train_sentences= []
with open('full_text_short.txt') as file:
    for line in file:
        st = line.rstrip()
        new_string = re.sub('[^a-zA-Z0-9 \n\.]', '', st)
        train_sentences.append(new_string)


In [ ]:
# Define your sentence transformer model using CLS pooling
model_name = 'roberta-base'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2022-01-05 16:37:28 - Use pytorch device: cuda


In [ ]:
train_sentences

In [ ]:
train_sentences = []
with open('full_text_short.txt') as file:
    for line in file:
      train_sentences.append(line.rstrip())
        # print("_____________________________________________________________")
        # print(line.rstrip())

In [ ]:
len(train_sentences)

7641

In [ ]:

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)



2022-01-05 17:35:17 - When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.


Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.3.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.9.crossattention.self.key.weight', 'roberta.encoder.layer.11.crossattention.self.key.weight', 'roberta.encoder.layer.5.crossattention.output.dense.weight', 'roberta.encoder.layer.7.crossattention.self.key.bias', 'roberta.encoder.layer.11.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.5.crossattention.self.query.bias', 'roberta.encoder.layer.6.crossattention.self.value.weight', 'roberta.encoder.layer.7.crossattention.self.value.bias', 'roberta.encoder.layer.9.crossattention.self.value.bias', 'roberta.encoder.layer.2.crossattention.self.value.bias', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.self.key.bias', 'roberta.encoder.layer.6.cross

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/105 [00:00<?, ?it/s]

In [ ]:
model.save('TSDAE64_1_kpm_data')

2022-01-05 17:36:57 - Save model to TSDAE64_1_kpm_data


In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
from datetime import datetime

if len(sys.argv) < 3:
    print("Usage: python train_mlm.py model_name data/train_sentences.txt [data/dev_sentences.txt]")
    exit()

model_name = "roberta-base"
per_device_train_batch_size = 16

save_steps = 1000               #Save model every 1k steps
num_train_epochs = 3            #Number of epochs
use_fp16 = False                #Set to True, if your GPU supports FP16 operations
max_length = 100                #Max length for a text input
do_whole_word_mask = True       #If set to true, whole words are masked
mlm_prob = 0.15                 #Probability that a word is replaced by a [MASK] token

# Load the model
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


output_dir = "output/roberta_base_mlm5_8"
print("Save checkpoints to:", output_dir)


##### Load our training datasets

train_sentences = []
with open('full_text_short.txt') as file:
    for line in file:
      train_sentences.append(line.rstrip())

print("Train sentences:", len(train_sentences))


#A dataset wrapper, that tokenizes our data on-the-fly
class TokenizedSentencesDataset:
    def __init__(self, sentences, tokenizer, max_length, cache_tokenization=False):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.max_length = max_length
        self.cache_tokenization = cache_tokenization

    def __getitem__(self, item):
        if not self.cache_tokenization:
            return self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)

        if isinstance(self.sentences[item], str):
            self.sentences[item] = self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)
        return self.sentences[item]

    def __len__(self):
        return len(self.sentences)

train_dataset = TokenizedSentencesDataset(train_sentences, tokenizer, max_length)



##### Training arguments

if do_whole_word_mask:
    data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
else:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    # evaluation_strategy="steps" if dev_dataset is not None else "no",
    per_device_train_batch_size=per_device_train_batch_size,
    eval_steps=save_steps,
    save_steps=save_steps,
    logging_steps=save_steps,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=use_fp16
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)



Save checkpoints to: output/roberta_base_mlm5_8
Train sentences: 7641


In [ ]:
train_sentences

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
model_name = 'roberta-base'
word_embedding_model = models.Transformer(model_name, max_seq_length=32)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Convert train sentences to sentence pairs
train_data = [InputExample(texts=[s, s]) for s in train_sentences]

# DataLoader to batch your data
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.MultipleNegativesRankingLoss(model)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    show_progress_bar=True
)

model.save('output/simcse-model8_16')

In [ ]:
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer
import logging
from datetime import datetime
import gzip
import sys
import tqdm

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

## Training parameters
model_name = 'roberta-base'
batch_size = 16
pos_neg_ratio = 8   # batch_size must be devisible by pos_neg_ratio
num_epochs = 1
max_seq_length = 75



# Save path to store our model
output_name = ''

model_output_path = 'output/train_ct{}-{}'.format(model_name, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))


# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

################# Read the train corpus  #################
# train_sentences = []
# with gzip.open(filepath, 'rt', encoding='utf8') if filepath.endswith('.gz') else open(filepath, encoding='utf8') as fIn:
#     for line in tqdm.tqdm(fIn, desc='Read file'):
#         line = line.strip()
#         if len(line) >= 10:
#             train_sentences.append(line)


logging.info("Train sentences: {}".format(len(train_sentences)))

# For ContrastiveTension we need a special data loader to construct batches with the desired properties
train_dataloader =  losses.ContrastiveTensionDataLoader(train_sentences, batch_size=batch_size, pos_neg_ratio=pos_neg_ratio)

# As loss, we losses.ContrastiveTensionLoss
train_loss = losses.ContrastiveTensionLoss(model)


warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          optimizer_params={'lr': 5e-5},
          checkpoint_path=model_output_path,
          show_progress_bar=True,
          use_amp=False  # Set to True, if your GPU supports FP16 cores
          )

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

2022-01-05 16:07:27 - Use pytorch device: cuda
2022-01-05 16:07:27 - Train sentences: 7641
2022-01-05 16:07:27 - Warmup-steps: 24


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/238 [00:00<?, ?it/s]

2022-01-05 16:09:06 - Save model to output/train_ctroberta-base-2022-01-05_16-07-09/238


In [ ]:
model.save('output/ct1_16')

2022-01-05 16:09:17 - Save model to output/ct1_16


In [ ]:
cd /content/drive/MyDrive/EMNLP_folder_4/dataset


/content/drive/MyDrive/EMNLP_folder_4/dataset


In [ ]:
import pandas as pd

In [ ]:
arg_df = pd.read_csv("arguments_train.csv")
kp_df = pd.read_csv("key_points_train.csv")
kp_df

,key_point_id,key_point,topic,stance
0,kp_0_0,Assisted suicide gives dignity to the person t...,Assisted suicide should be a criminal offence,-1
1,kp_0_1,Assisted suicide reduces suffering,Assisted suicide should be a criminal offence,-1
2,kp_0_2,People should have the freedom to choose to en...,Assisted suicide should be a criminal offence,-1
3,kp_0_3,The terminally ill would benefit from assisted...,Assisted suicide should be a criminal offence,-1
4,kp_0_4,Assisted suicide allows people to solicit some...,Assisted suicide should be a criminal offence,1
...,...,...,...,...
202,kp_27_3,subsidizing vocational education is expensive,We should subsidize vocational education,-1
203,kp_27_4,subsidizing vocational education promotes thos...,We should subsidize vocational education,1
204,kp_27_5,vocational education is a good career choice,We should subsidize vocational education,1
205,kp_27_6,vocational education better fits many students,We should subsidize vocational education,1


In [ ]:
import re
train_sentences= []
for index, row in arg_df.iterrows():
    arg = row["argument"]
    arg_sent = arg.rstrip()
    clean = re.sub('[^a-zA-Z0-9 \n\.]', '', arg_sent)
    tok = nltk.tokenize.sent_tokenize(clean)
    for el in tok: 
      train_sentences.append(el)
    # train_sentences.append(new_string)
for index,row in kp_df.iterrows():
    arg = row["key_point"]
    arg_sent = arg.rstrip()
    clean = re.sub('[^a-zA-Z0-9 \n\.]', '', arg_sent)
    tok = nltk.tokenize.sent_tokenize(clean)
    for el in tok: 
      train_sentences.append(el)

topics= set()
for index, row in arg_df.iterrows():
  topics.add(row["topic"])
for el in topics: 
  train_sentences.append(el)



In [ ]:
len(train_sentences)

6667

In [ ]:
len(train_sentences)

208

In [ ]:
len(train_sentences)

6667

In [ ]:
test = train_sentences[0]
tok = nltk.tokenize.sent_tokenize(test)

In [ ]:
tok

['people reach their limit when it comes to their quality of life and should be able to end their suffering.',
 'this can be done with little or no suffering by assistance and the person is able to say good bye.']